# IE 306.02 Assignment 1
In this assignment we have implemented a call center simulation. Clients place a call to front desk operator who then further forwards them to an expert operator. Clients may renage after front desk operator while waiting for expert, expert takes a break sometimes and both queues are FCFS. Exact details of both queues can be found in assignment description.

We have designed the system as two M/M/1 queues put one after another. In the first queue, server is front desk opetor who we modeled as a resource in the SimPy environment. Similarly, expert is the server in the second queue who is also modeled as a resource in SimPy. 

Below, we first describe our code and later interpret the statistics and results we have gathered from the simulation.

First we import the basic libraries.

In [1]:
import simpy
import random
import numpy.random
import numpy as np
from math import log,sqrt
import itertools # for easy list operations
from collections import namedtuple

A little helper function for formatting seconds to desired time format.

In [2]:
def format_time(total_minutes):
    secs = int((total_minutes % 1)*60)
    mins = int(total_minutes % 60)
    hours = int(total_minutes //60) % 24
    days = int(total_minutes//60//24)
    return f" {days:02}:{hours:02}:{mins:02}:{secs:02}"
        
# an object to hold expert queue statistics
Expert_Q_Stat = namedtuple('Expert_Q_Stat', ['enter_e_q', 'exit_e_q'])

Below, we first describe our code and then interpret results.

We ollect and report statistics on: 

* Utilization of the front-desk operator

* Utilization of the expert operators, 

* Average Total Waiting Time,

* `Maximum Total Waiting Time` to `Total System Time Ratio`,

* Average number of people waiting to be served by the expert operator.

Global constants that will be used in the code.

In [3]:
# interarrival rates are exponentially distributed with mean 14.3 min
INTERARRIVAL_MEAN = 14.3 
# service time of first operator is LogNormal distributed with mean 7.2 and sd 2.7
m = 7.2
v = 2.7

FIRST_SERVICE_MEAN = log((m**2)/sqrt(v+m**2))
FIRST_SERVICE_STD = sqrt(log(v/(m**2)+1))


# customers leave queue if not serviced in exponentially distributed time with mean 60 min
RENEGE_TIME_MEAN = 60
# service time of expert is exponentially distributed with mean 10.2
EXPERT_SERVICE_MEAN = 10.2
# number of breaks taken by expert is Poisson distributed with mean 8 breaks
BREAKS_MEAN = 8
BREAK_DUR = 3

# number of customers simulation will be run for
CALLNUM1=1000
CALLNUM2=5000

LAST_EXIT = 0

LAST_CUSTOMER = 1

CUSTOMER_NUM=CALLNUM1


#RANDOM_SEED = 971
#random.seed(RANDOM_SEED)

Here, we define the necessary set of arrays for bookkeeping

In [4]:
# service times for front-desk operator
first_service_times = [] 
# service times for expert 
second_service_times = []
# time spent by a customer while it waits for the operator (Queue waiting time Wq)
first_queue_w_times = [] 
second_queue_w_times = [] 


expert_queue_log = []

# counter for breaks taken by the expert
break_count = 0



A said above, we modeled the front desk and expert operators as resources, calls(called customer in code) and breaks that expert takes are modeled as processes. Breaks taken by the expert makes the expert resource unavailable for 3 mins, so it made sense to model breaks as processes that try to get expert resource.

Class definition for breaks. Breaks are generated for expert operator. When they happen, they take 3 minutes long and expert is not available at that time. 

In [5]:
class Break:
    def __init__(self, env, expert):
        self.env = env
        self.expert = expert
        # it only action is happen, which can be thought as just being initiated
        self.action = env.process(self.happen())
    
    
    def happen(self):
        print(f"Break is on the way \t\t\t\t at {format_time(self.env.now)}")
        with self.expert.request() as req:
            yield req
            print(f"Break started \t\t\t\t\t at {format_time(self.env.now)}")
            yield self.env.timeout(BREAK_DUR)
            print(f"Break ended \t\t\t\t\t at {format_time(self.env.now)}")

* The class definition for the customers arriving at the modeled system. When they are created, they immediatelly initiate a call (i.e. activate the call process). 

* Once a call is initiated, this is registered as a request to the operator resource. The customer is put on hold until the resource activates it back. 

* When the resource is available, the customer is activated and it then initiates first the ask_question_operator process. The duration of a question-answer session is determined randomly according to a LogNormal distribution.

* After this, they are forwarded to expert operator. While waiting for the expert, they may renege after some time taken from an exponential distribution if they aren't addressed by the expert.

* Service time of the expert is taken from another exponential distribution.


In [6]:
class Customer(object):
    def __init__(self, _id, env, operator, expert):
        self.env = env
        self.id = _id
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
    
    
    def call(self):
        print(f"Cust {self.id:4} entered the \t\t FRONT Queue \t at {format_time(self.arrival_t)}")
 
        with operator.request() as op_req :
            yield op_req
            self.wait_front = self.env.now - self.arrival_t
            print(f"Cust {self.id:4} started talking with \t FRONT Desk \t at {format_time(self.env.now)}")
            if self.wait_front:
                    print(f"Cust {self.id:4} waited in \t\t FRONT Queue \t for{format_time(self.wait_front)}")
            first_queue_w_times.append(self.wait_front)
            yield self.env.process(self.ask_question_operator())
            self.finish_operator=self.env.now
            print(f"Cust {self.id:4} entered the \t\t EXPERT Queue \t at {format_time(self.finish_operator)}")
        

        with expert.request() as exp_req: 
            enter_e_q = self.env.now
            # customers may renege in the expert queue
            self.patience=random.expovariate(1/RENEGE_TIME_MEAN)
            results = yield exp_req | env.timeout(self.patience)
            self.wait_exp=self.env.now-self.finish_operator
            second_queue_w_times.append(self.wait_exp) 
            exit_e_q = self.env.now
            global expert_queue_log
            expert_queue_log.append(Expert_Q_Stat(enter_e_q, exit_e_q))

            if exp_req not in results:
                #Leave queue, reneged
                print(f"Cust {self.id:4} exited \t\t SUDDENLY \t at {format_time(self.env.now)}")
                print(f"-- after waiting {format_time(self.wait_exp)}")

            else:
            # We got to the expert
                print(f"Cust {self.id:4} started talking with \t EXPERT \t at {format_time(self.env.now)}")
                if self.wait_exp:
                    print(f"Cust {self.id:4} waited in \t\t EXPERT Queue \t for{format_time(self.wait_exp)}")
                yield self.env.process(self.ask_question_expert())
                self.finish_expert=self.env.now
                print(f"Cust {self.id:4} exited \t\t NORMALLY \t at {format_time(self.finish_expert)}")
        
        global LAST_EXIT
        LAST_EXIT = self.env.now
            
    def ask_question_operator(self):
        service_opr = random.lognormvariate(FIRST_SERVICE_MEAN,FIRST_SERVICE_STD)
        first_service_times.append(service_opr)
        yield self.env.timeout(service_opr)


    def ask_question_expert(self):
        service_exp = random.expovariate(1/EXPERT_SERVICE_MEAN)
        second_service_times.append(service_exp)
        yield self.env.timeout(service_exp)

Generator functions for customers and breaks. 

* Interrarrival times for both are taken from different exponential distributions. 

* Customer generator runs as many times as the number of total customers we want in our simulation, break generator runs through the whole shift.


In [7]:
def customer_generator(env, operator,expert,callnum):
    for i in range(callnum):
        global LAST_CUSTOMER
        LAST_CUSTOMER = LAST_CUSTOMER + 1
        #if env.now >= 60*8:
        #    break
        # lamda is how many for any minute(rate). 1/lamda is how many min for 1 event
        time_between_next_customer = random.expovariate(1/INTERARRIVAL_MEAN)
        customer = Customer(i+1, env, operator,expert)
        #if env.now + time_between_next_customer >= 60*8:
        #    break
        yield env.timeout(time_between_next_customer)
        
def break_generator(env, expert):
    while LAST_CUSTOMER<=CUSTOMER_NUM:
        time_to_break = random.expovariate(1/60)
        #if env.now + time_to_break >= 60*8:
        #    break
        yield env.timeout(time_to_break)
        Break(env, expert)
        global break_count
        break_count += 1

Here, we define our processes and resources and start running the simulation environment!

# To run the simulation for 1000 customers, make sure that CUSTOMER_NUM equals CALLNUM1

In [8]:
env = simpy.Environment()
operator = simpy.Resource(env, capacity = 1)
expert=simpy.Resource(env, capacity = 1)
env.process(customer_generator(env, operator,expert,CUSTOMER_NUM))
env.process(break_generator(env, expert))
end_simulation = simpy.Event(env)
env.run() 

Cust    1 entered the 		 FRONT Queue. 	 at  00:00:00:00
Cust    1 started talking with 	 FRONT Desk 	 at  00:00:00:00
Cust    2 entered the 		 FRONT Queue. 	 at  00:00:00:19
Cust    3 entered the 		 FRONT Queue. 	 at  00:00:01:13
Cust    4 entered the 		 FRONT Queue. 	 at  00:00:07:49
Cust    1 entered the 		 EXPERT Queue. 	 at  00:00:09:47
Cust    2 started talking with 	 FRONT Desk 	 at  00:00:09:47
Cust    2 waited in 		 FRONT Queue 	 for 00:00:09:27
Cust    1 started talking with 	 EXPERT 	 at  00:00:09:47
Cust    1 exited 		 NORMALLY 	 at  00:00:11:14
Cust    5 entered the 		 FRONT Queue. 	 at  00:00:11:55
Cust    2 entered the 		 EXPERT Queue. 	 at  00:00:15:01
Cust    3 started talking with 	 FRONT Desk 	 at  00:00:15:01
Cust    3 waited in 		 FRONT Queue 	 for 00:00:13:47
Cust    2 started talking with 	 EXPERT 	 at  00:00:15:01
Cust    6 entered the 		 FRONT Queue. 	 at  00:00:16:37
Cust    2 exited 		 NORMALLY 	 at  00:00:20:48
Cust    3 entered the 		 EXPERT Queue. 	 at  00:

Break is on the way 				 at  01:12:01:34
Cust  165 entered the 		 EXPERT Queue. 	 at  01:12:07:20
Cust  166 started talking with 	 FRONT Desk 	 at  01:12:07:20
Cust  166 waited in 		 FRONT Queue 	 for 00:00:06:00
Cust  167 entered the 		 FRONT Queue. 	 at  01:12:12:07
Cust  166 entered the 		 EXPERT Queue. 	 at  01:12:12:57
Cust  167 started talking with 	 FRONT Desk 	 at  01:12:12:57
Cust  167 waited in 		 FRONT Queue 	 for 00:00:00:49
Cust  166 exited 		 SUDDENLY 	 at  01:12:14:58
-- after waiting  00:00:02:01
Cust  164 exited 		 NORMALLY 	 at  01:12:18:02
Break started 					 at  01:12:18:02
Break ended 					 at  01:12:21:02
Cust  165 started talking with 	 EXPERT 	 at  01:12:21:02
Cust  165 waited in 		 EXPERT Queue 	 for 00:00:13:42
Cust  167 entered the 		 EXPERT Queue. 	 at  01:12:22:01
Cust  168 entered the 		 FRONT Queue. 	 at  01:12:22:07
Cust  168 started talking with 	 FRONT Desk 	 at  01:12:22:07
Cust  165 exited 		 NORMALLY 	 at  01:12:24:15
Cust  167 started talking with 

Cust  348 started talking with 	 FRONT Desk 	 at  03:06:10:07
Cust  348 waited in 		 FRONT Queue 	 for 00:00:06:22
Cust  347 started talking with 	 EXPERT 	 at  03:06:10:07
Cust  348 entered the 		 EXPERT Queue. 	 at  03:06:16:13
Cust  349 started talking with 	 FRONT Desk 	 at  03:06:16:13
Cust  349 waited in 		 FRONT Queue 	 for 00:00:12:22
Cust  347 exited 		 NORMALLY 	 at  03:06:21:24
Cust  348 started talking with 	 EXPERT 	 at  03:06:21:24
Cust  348 waited in 		 EXPERT Queue 	 for 00:00:05:11
Cust  351 entered the 		 FRONT Queue. 	 at  03:06:21:38
Cust  352 entered the 		 FRONT Queue. 	 at  03:06:23:25
Cust  349 entered the 		 EXPERT Queue. 	 at  03:06:24:10
Cust  350 started talking with 	 FRONT Desk 	 at  03:06:24:10
Cust  350 waited in 		 FRONT Queue 	 for 00:00:16:19
Cust  348 exited 		 NORMALLY 	 at  03:06:26:50
Cust  349 started talking with 	 EXPERT 	 at  03:06:26:50
Cust  349 waited in 		 EXPERT Queue 	 for 00:00:02:40
Cust  349 exited 		 NORMALLY 	 at  03:06:27:07
Cust  

Cust  518 started talking with 	 FRONT Desk 	 at  04:17:29:10
Break is on the way 				 at  04:17:36:00
Cust  518 entered the 		 EXPERT Queue. 	 at  04:17:36:32
Cust  519 entered the 		 FRONT Queue. 	 at  04:17:36:41
Cust  519 started talking with 	 FRONT Desk 	 at  04:17:36:41
Cust  511 exited 		 SUDDENLY 	 at  04:17:37:36
-- after waiting  00:01:01:38
Cust  520 entered the 		 FRONT Queue. 	 at  04:17:41:47
Cust  507 exited 		 NORMALLY 	 at  04:17:41:50
Break started 					 at  04:17:41:50
Cust  518 exited 		 SUDDENLY 	 at  04:17:43:03
-- after waiting  00:00:06:30
Cust  519 entered the 		 EXPERT Queue. 	 at  04:17:43:50
Cust  520 started talking with 	 FRONT Desk 	 at  04:17:43:50
Cust  520 waited in 		 FRONT Queue 	 for 00:00:02:02
Break ended 					 at  04:17:44:50
Cust  509 started talking with 	 EXPERT 	 at  04:17:44:50
Cust  509 waited in 		 EXPERT Queue 	 for 00:01:27:28
Break is on the way 				 at  04:17:46:12
Break is on the way 				 at  04:17:50:24
Cust  520 entered the 		 EXPE

Cust  673 entered the 		 FRONT Queue. 	 at  06:04:36:49
Break is on the way 				 at  06:04:43:14
Break started 					 at  06:04:43:14
Cust  672 entered the 		 EXPERT Queue. 	 at  06:04:44:50
Cust  673 started talking with 	 FRONT Desk 	 at  06:04:44:50
Cust  673 waited in 		 FRONT Queue 	 for 00:00:08:01
Break ended 					 at  06:04:46:14
Cust  672 started talking with 	 EXPERT 	 at  06:04:46:14
Cust  672 waited in 		 EXPERT Queue 	 for 00:00:01:23
Cust  673 entered the 		 EXPERT Queue. 	 at  06:04:52:06
Cust  674 entered the 		 FRONT Queue. 	 at  06:04:57:05
Cust  674 started talking with 	 FRONT Desk 	 at  06:04:57:05
Cust  674 entered the 		 EXPERT Queue. 	 at  06:05:02:14
Cust  675 entered the 		 FRONT Queue. 	 at  06:05:04:21
Cust  675 started talking with 	 FRONT Desk 	 at  06:05:04:21
Cust  674 exited 		 SUDDENLY 	 at  06:05:04:48
-- after waiting  00:00:02:33
Cust  676 entered the 		 FRONT Queue. 	 at  06:05:09:08
Cust  675 entered the 		 EXPERT Queue. 	 at  06:05:13:40
Cust  676

Cust  867 waited in 		 FRONT Queue 	 for 00:00:09:28
Cust  868 entered the 		 FRONT Queue. 	 at  08:01:22:37
Cust  869 entered the 		 FRONT Queue. 	 at  08:01:23:04
Cust  870 entered the 		 FRONT Queue. 	 at  08:01:24:54
Cust  867 entered the 		 EXPERT Queue. 	 at  08:01:28:49
Cust  868 started talking with 	 FRONT Desk 	 at  08:01:28:49
Cust  868 waited in 		 FRONT Queue 	 for 00:00:06:12
Cust  871 entered the 		 FRONT Queue. 	 at  08:01:29:29
Cust  866 exited 		 SUDDENLY 	 at  08:01:31:48
-- after waiting  00:00:10:46
Cust  872 entered the 		 FRONT Queue. 	 at  08:01:32:05
Cust  868 entered the 		 EXPERT Queue. 	 at  08:01:33:56
Cust  869 started talking with 	 FRONT Desk 	 at  08:01:33:56
Cust  869 waited in 		 FRONT Queue 	 for 00:00:10:51
Cust  867 exited 		 SUDDENLY 	 at  08:01:36:10
-- after waiting  00:00:07:21
Cust  869 entered the 		 EXPERT Queue. 	 at  08:01:38:44
Cust  870 started talking with 	 FRONT Desk 	 at  08:01:38:44
Cust  870 waited in 		 FRONT Queue 	 for 00:00:13:

Lastly, we print our statistics we gathered from the simulation

In [17]:
util_front = sum(first_service_times) / LAST_EXIT
util_exp = sum(second_service_times) / LAST_EXIT

total_waits = [f_w + e_w for f_w, e_w in itertools.zip_longest(
    first_queue_w_times, 
    second_queue_w_times, 
    fillvalue=0
)]

total_services = [f_s + e_s for f_s, e_s in itertools.zip_longest(
    first_service_times,
    second_service_times,
    fillvalue=0
)]

total_system = [w + s for w, s in itertools.zip_longest(
    total_waits,
    total_services,
    fillvalue=0
)]

# this makes sure everything went smoothly 
assert(len(total_waits) == len(total_system))

max_ratio = max((w/sys if sys else 0 for w, sys in zip(total_waits, total_system)))

# a helper function to calculate avg length of expert queue
def log_to_avg_len(log):
   ins = sorted((entry.enter_e_q for entry in log))
   outs = sorted((entry.exit_e_q for entry in log))
   significant_points = np.array(sorted(ins + outs))
   in_mask = np.array([i in ins for i in significant_points])
   out_mask = np.array([o in outs for o in significant_points])
   queue_len = np.zeros(len(significant_points))

   
   for i, is_in in enumerate(in_mask):
      if is_in:
         queue_len[i:] += 1

   for i, is_in in enumerate(out_mask):
      if is_in:
         queue_len[i:] -= 1

   timedeltas = np.copy(significant_points)
   for i, elem in enumerate(timedeltas):
      timedeltas[i+1:] -= elem

   return np.sum( timedeltas*queue_len ) / outs[-1]

stats = {
    "break_count": break_count,
    "util_front": util_front,
    "util_exp": util_exp,
    "avg_total_wait": sum(total_waits) / len(total_waits),
    "max_wait_2_sys_ratio": max_ratio,
    "avg_len_of_exp_q": log_to_avg_len(expert_queue_log),
}

print(f"Number of breaks taken: {stats['break_count']}")
print(f"Utilization of front desk operator: {stats['util_front']}")
print(f"Utilization of expert operator: {stats['util_exp']}")
print(f"Average total waiting time: {format_time(stats['avg_total_wait'])}")
print(f"Maximum of total waiting time to system time ratios: {stats['max_wait_2_sys_ratio']:.2f}")
print(f"Average length of expert waiting queue: {stats['avg_len_of_exp_q']}")


Number of breaks taken: 236
Utilization of front desk operator: 0.5375077047218247
Utilization of expert operator: 0.6323212547201499
Average total waiting time:  00:00:13:09
Maximum of total waiting time to system time ratios: 0.93
Average length of expert waiting queue: 0.7884534999132871


In [10]:
print (len(first_queue_w_times))

1000


In [11]:
print (len(second_queue_w_times))

1000


In [12]:
print(format_time(env.now))
print(format_time(LAST_EXIT))

 09:09:57:58
 09:09:07:58
